In [ ]:
#"""
!pip install pytorch-forecasting
!pip install webdataset 
!pip install torchvision
!pip install pytorch-lightning
!pip install torchmetrics
#"""

In [ ]:
import torch
import logging
import os, warnings, copy
from pathlib import Path
import numpy as np
import pandas as pd
import webdataset as wds
import torch.nn as nn
import torchvision.transforms as transforms
from torchmetrics.functional import accuracy
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

In [ ]:
class LSTM(pl.LightningModule):
  def __init__(self, n_features, n_hidden, n_layers, seq_len, dropout=0.5, lr=1e-3, batch_size=40, num_epochs=100):
    super(LSTM, self).__init__()
    self.n_features = n_features
    self.n_hidden = n_hidden
    self.n_layers = n_layers
    self.dropout = dropout
    self.seq_len = seq_len
    self.lr = lr
    self.criterion = nn.MSELoss()
    self.batch_size = batch_size
    self.num_epochs = num_epochs
    self.__build__()

  def __build__(self):
    self.lstm = nn.LSTM(
        input_size = self.n_features,
        hidden_size = self.n_hidden,
        num_layers = self.n_layers,
        dropout = self.dropout,
        batch_first = True
    )
    self.fc = nn.Linear(
        in_features=self.n_hidden, 
        out_features=self.n_features
    )
  
  def reset_hidden_state(self):
    self.hidden = (torch.zeros(self.n_layers, self.seq_len, self.n_hidden) for i in range(self.n_layers))

  def reshape_batch(self, x):
    ECG_x = x[:,:,0]
    EDA_x = x[:,:,1]
    EMG_x = x[:,:,2]
    Temp_x = x[:,:,3]
    Resp_x = x[:,:,4]
    ACC_x_x = x[:,:,5]
    ACC_y_x = x[:,:,6]
    ACC_z_x = x[:,:,7]
    
    ECG_x = torch.flatten(ECG_x, 1)
    EDA_x = torch.flatten(EDA_x, 1)
    EMG_x = torch.flatten(EMG_x, 1)
    Temp_x = torch.flatten(Temp_x, 1)
    Resp_x = torch.flatten(Resp_x, 1)
    ACC_x_x = torch.flatten(ACC_x_x, 1)
    ACC_y_x = torch.flatten(ACC_y_x, 1)
    ACC_z_x = torch.flatten(ACC_z_x, 1)
    
    x = torch.cat((ECG_x,EDA_x,EMG_x,Temp_x,Resp_x,ACC_x_x,ACC_y_x,ACC_z_x), dim=1)
    return x

  def forward(self, x):
    x = x.float()
    x = self.reshape_batch(x)
    print(f"input shape: {x.shape}")
    self.lstm.flatten_parameters()
    _, (hidden, _) = self.lstm(x)
    out = hidden[-1]
    return self.fc(out)

  def train_dataloader(self):
    dataset = (
      wds.WebDataset('gdrive/MyDrive/emotion_forecasting/WESAD_RespiBAN_binary_train.tar')
        .decode()
        .to_tuple("input.npy", "label.cls")
        .map_tuple(transforms.ToTensor(), lambda x: 0 if x == 1 else 1 if x == 2 else 2 )
    )

    return torch.utils.data.DataLoader(dataset, batch_size=self.batch_size)

  def test_dataloader(self):
    dataset = (
      wds.WebDataset('gdrive/MyDrive/emotion_forecasting/WESAD_RespiBAN_binary_test.tar')
        .decode()
        .to_tuple("input.npy", "label.cls")
        .map_tuple(transforms.ToTensor(), lambda x: 0 if x == 1 else 1 if x == 2 else 2 )
    )

    return torch.utils.data.DataLoader(dataset, batch_size=self.batch_size)

  def configure_optimizers(self):
    return torch.optim.AdamW(self.parameters(), lr=self.lr)

  def training_step(self, batch, batch_idx):
    inputs, _ = batch
    outputs = self.forward(inputs)
    loss = self.criterion(outputs, inputs)

    return dict(loss = loss)

  def test_step(self, batch, batch_idx):
    inputs, labels = batch
    outputs = self.forward(inputs)
    acc = accuracy(outputs, labels)

    self.log("accuracy:", acc, prog_bar=True)


In [ ]:
BATCH_SIZE = 40
NUM_EPOCHS = 100

biomarker_forecast_model = LSTM(
    n_features=3500,
    n_hidden=120,
    n_layers=3,
    seq_len=300,
    dropout=0.2,
    lr=1e-3,
    batch_size=BATCH_SIZE,
    num_epochs=NUM_EPOCHS
)

trainer = pl.Trainer(max_epochs=NUM_EPOCHS, accelerator="auto", auto_lr_find=True)
logging.getLogger("lightning").setLevel(logging.WARNING)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

trainer.fit(biomarker_forecast_model)

Missing logger folder: /content/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type    | Params
--------------------------------------
0 | criterion | MSELoss | 0     
1 | lstm      | LSTM    | 2.0 M 
2 | fc        | Linear  | 423 K 
--------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
9.578     Total estimated model params size (MB)


Mounted at /content/gdrive/


Training: 0it [00:00, ?it/s]

inputs shape: torch.Size([40, 1, 8, 3500])
input shape: torch.Size([40, 28000])


RuntimeError: ignored